<a href="https://colab.research.google.com/github/prajeet26/NER-LLM/blob/main/Task1_2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df

data = []
with open("/content/drive/MyDrive/FLAT_RCL.txt", "r") as doc:
    for line in doc:
        fields = line.strip().split("\t")
        data.append(fields)

column_names = [
    'RECORD_ID', 'CAMPNO', 'MAKETXT', 'MODELTXT', 'YEARTXT',
    'MFGCAMPNO', 'COMPNAME', 'MFGNAME', 'BGMAN', 'ENDMAN',
    'RCLTYPECD', 'POTAFF', 'ODATE', 'INFLUENCED_BY', 'MFGTXT',
    'RCDATE', 'DATEA', 'RPNO', 'FMVSS', 'DESC_DEFECT',
    'CONEQUENCE_DEFECT', 'CORRECTIVE_ACTION', 'NOTES',
    'RCL_CMPT_ID', 'MFR_COMP_NAME', 'MFR_COMP_DESC', 'MFR_COMP_PTNO'
]
df = pd.DataFrame(data, columns=column_names)


In [65]:
new_df = df[['DESC_DEFECT', 'CONEQUENCE_DEFECT', 'CORRECTIVE_ACTION', 'NOTES']]


In [4]:
new_df

,DESC_DEFECT,CONEQUENCE_DEFECT,CORRECTIVE_ACTION,NOTES
0,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...
1,CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC...,"THIS, IN TURN, COULD CAUSE THE BATTERY CABLES ...",DEALERS WILL INSPECT THE BATTERY CABLES FOR TH...,ALSO CONTACT THE NATIONAL HIGHWAY TRAFFIC SAFE...
2,"ON CERTAIN FOLDING TENT CAMPERS, THE FEDERAL C...","IF THE TIRES WERE INFLATED TO 80 PSI, THEY COU...",OWNERS WILL BE MAILED CORRECT LABELS FOR INSTA...,"ALSO, CUSTOMERS CAN CONTACT THE NATIONAL HIGHW..."
3,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...
4,"ON CERTAIN CLASS A MOTOR HOMES, THE FLOOR TRUS...",CONDITIONS CAN RESULT IN THE BOTTOMING OUT THE...,DEALERS WILL INSPECT THE FLOOR TRUSS NETWORK S...,CUSTOMERS CAN ALSO CONTACT THE NATIONAL HIGHWA...
...,...,...,...,...
255863,Volvo Bus Corporation (Volvo Bus) is recalling...,A detached link rod can cause a loss of steeri...,Owners are advised not to drive their vehicle ...,Owners may also contact the National Highway T...
255864,"Daimler Trucks North America, LLC (DTNA) is re...",Difficulty steering and a loss of directional ...,The remedy is currently under development. Ow...,Owners may also contact the National Highway T...
255865,"Daimler Trucks North America, LLC (DTNA) is re...",Difficulty steering and a loss of directional ...,The remedy is currently under development. Ow...,Owners may also contact the National Highway T...
255866,"Daimler Trucks North America, LLC (DTNA) is re...",Difficulty steering and a loss of directional ...,The remedy is currently under development. Ow...,Owners may also contact the National Highway T...


In [5]:
new_df.shape[0]

255868

In [6]:
!pip install -q -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.6 MB/s eta 0:00:00


In [7]:
!pip install -q -U transformers
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.5 MB/s eta 0:00:00


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

4bit for lesser memory footprint

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    disable_exllama=True
)

In [10]:
!pip install auto-gptq
!pip install optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.1/407.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.2 MB/s eta 0:00:00


In [11]:
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

In [12]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [13]:
small_df = df.sample(n=1500)
small_df


,RECORD_ID,CAMPNO,MAKETXT,MODELTXT,YEARTXT,MFGCAMPNO,COMPNAME,MFGNAME,BGMAN,ENDMAN,...,RPNO,FMVSS,DESC_DEFECT,CONEQUENCE_DEFECT,CORRECTIVE_ACTION,NOTES,RCL_CMPT_ID,MFR_COMP_NAME,MFR_COMP_DESC,MFR_COMP_PTNO
23815,23816,90V140000,MACK,MS250,1988,,STEERING:RACK AND PINION,RVI INC.,19881201,19900401,...,,,CAPSCREW THAT ATTACHES STEERING GEAR TO THE ST...,THIS COULD AFFECT STEERING CONTROL AND RESULT ...,REPLACE THE STEERING BRACKET AND CAPSCREWS.,SYSTEM: POWER STEERING GEAR MOUNTING.VEHICLE D...,000005951000057432000000008,None,None,None
218615,218616,22V365000,MERCEDES-BENZ,GLS450,2020,2022060005,ELECTRICAL SYSTEM:SOFTWARE,"Mercedes-Benz USA, LLC",,,...,,,"Mercedes-Benz USA, LLC (MBUSA) is recalling ce...",A disabled eCall system would prevent a vehicl...,The communication module software will be upda...,Owners may also contact the National Highway T...,000120389004181720000001105,SW communication module,SW communication module,A2229024021
73767,73768,09V458000,FCCC,B2,2008,FL-567,SUSPENSION:REAR,DAIMLER TRUCKS NORTH AMERICA,20070601,20090928,...,,,DAIMLER TRUCKS IS RECALLING CERTAIN MODEL YEAR...,LOOSE OR MISSING FASTENERS MAY ALLOW THE REAR ...,DEALERS WILL INSPECT AND ADJUST THE FASTENERS ...,DAILMER TRUCKS CAMPAIGN NUMBER IS FL-567.OWNER...,000034078000252583000000045,None,None,None
124326,124327,16V840000,MASERATI,QUATTROPORTE,2017,324,TIRES:SIDEWALL,"Maserati North America, Inc.",20160229,20161029,...,,,"Maserati North America, Inc. (Maserati) is rec...",A loss of air may result in tire failure or a ...,"Maserati will notify owners, and dealers will ...",Owners may also contact the National Highway T...,000062547002192740000000302,None,None,None
243720,243721,23V758000,FORD,F-350 SD,2022,23S59,POWER TRAIN:AXLE HUBS,Ford Motor Company,,,...,,,Ford Motor Company is recalling certain 2022-2...,Wheel detachment can result in a loss of vehic...,Owners are advised not to drive their vehicle ...,Owners may also contact the National Highway T...,000126077004679750000000199,Front Axle,F-450 / F-550 Front Axle - Dual Rear Wheel 4x2...,PC34-3001-CD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163041,163042,21V058000,MERCEDES-BENZ,SL63,2017,2021020025,COMMUNICATION: AUTO CRASH NOTIFICATION,"Mercedes-Benz USA, LLC",,,...,,,"Mercedes-Benz USA, LLC (MBUSA) is recalling ce...",Failure of the eCall system may result in emer...,"MBUSA will notify owners, and the communicatio...",Owners may also contact the National Highway T...,000108471002457728000001420,SW HERMES,Control Module Unit Software,A2139020305
227434,227435,22V365000,MERCEDES-BENZ,AMG GLS63,2019,2022060005,ELECTRICAL SYSTEM:SOFTWARE,"Mercedes-Benz USA, LLC",,,...,,,"Mercedes-Benz USA, LLC (MBUSA) is recalling ce...",A disabled eCall system would prevent a vehicl...,The communication module software will be upda...,Owners may also contact the National Highway T...,000120389004701764000001105,SW communication module,SW communication module,A2229023217
245040,245041,23V158000,HONDA,ACCORD HYBRID,2019,"NDA, QDB, BDC, MDD",SEAT BELTS:FRONT:BUCKLE ASSEMBLY,Honda (American Honda Motor Co.),,,...,,,Honda (American Honda Motor Co.) is recalling ...,An unlatched seat belt cannot properly restrai...,Dealers will replace the driver and front pass...,Owners may also contact the National Highway T...,000124882004143743000000268,Buckle Set L / Buckle Set R,Front Seat Belt,04816-THR-A00ZA
205501,205502,22V166000,NEWMAR,VENTANA,2021,,STEERING:CRITICAL FASTENERS,Newmar Corporation,,,...,,,Newmar Corporation (Newmar) is recalling certa...,Loss of steering control increases the risk of...,Dealers will inspect and tighten the upper and...,Owners may also contact the National Highway T...,000119149004709750000001651,Steering ColumnUpper Bolt/Nut,Bolt/Nut,None


In [14]:
new_df_small = small_df[['DESC_DEFECT', 'CONEQUENCE_DEFECT', 'CORRECTIVE_ACTION', 'NOTES']]
new_df_small


,DESC_DEFECT,CONEQUENCE_DEFECT,CORRECTIVE_ACTION,NOTES
23815,CAPSCREW THAT ATTACHES STEERING GEAR TO THE ST...,THIS COULD AFFECT STEERING CONTROL AND RESULT ...,REPLACE THE STEERING BRACKET AND CAPSCREWS.,SYSTEM: POWER STEERING GEAR MOUNTING.VEHICLE D...
218615,"Mercedes-Benz USA, LLC (MBUSA) is recalling ce...",A disabled eCall system would prevent a vehicl...,The communication module software will be upda...,Owners may also contact the National Highway T...
73767,DAIMLER TRUCKS IS RECALLING CERTAIN MODEL YEAR...,LOOSE OR MISSING FASTENERS MAY ALLOW THE REAR ...,DEALERS WILL INSPECT AND ADJUST THE FASTENERS ...,DAILMER TRUCKS CAMPAIGN NUMBER IS FL-567.OWNER...
124326,"Maserati North America, Inc. (Maserati) is rec...",A loss of air may result in tire failure or a ...,"Maserati will notify owners, and dealers will ...",Owners may also contact the National Highway T...
243720,Ford Motor Company is recalling certain 2022-2...,Wheel detachment can result in a loss of vehic...,Owners are advised not to drive their vehicle ...,Owners may also contact the National Highway T...
...,...,...,...,...
163041,"Mercedes-Benz USA, LLC (MBUSA) is recalling ce...",Failure of the eCall system may result in emer...,"MBUSA will notify owners, and the communicatio...",Owners may also contact the National Highway T...
227434,"Mercedes-Benz USA, LLC (MBUSA) is recalling ce...",A disabled eCall system would prevent a vehicl...,The communication module software will be upda...,Owners may also contact the National Highway T...
245040,Honda (American Honda Motor Co.) is recalling ...,An unlatched seat belt cannot properly restrai...,Dealers will replace the driver and front pass...,Owners may also contact the National Highway T...
205501,Newmar Corporation (Newmar) is recalling certa...,Loss of steering control increases the risk of...,Dealers will inspect and tighten the upper and...,Owners may also contact the National Highway T...


In [15]:
model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
  

In [16]:
dataset=new_df_small

In [17]:


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, dataset, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (1200, 4)
y_train shape: (1200, 4)
X_test shape: (300, 4)
y_test shape: (300, 4)


In [18]:
transformers.__version__

'4.38.2'

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Chain Of Thought Example

In [20]:
!pip install typing


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=40683e910ceeca52af79a81d9857aa47a8e7511f9ceebb3db79c6d95a57d3fb6
  Stored in directory: /root/.cache/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing


In [41]:
entlist = [
    "Manufacturer:like name of the car manufacturer",
    "Model: for example like  a model like focus from manufacturer ford",
    "Year: for example like made  in the  year 2000",
    "Recall_Number:for example like  02V288000",
    "Component: for example like ELECTRICAL SYSTEM or  BATTERY CABLES",
    "Issue_Defect: for example like CERTAIN PASSENGER VEHICLES EQUIPPED WITH ZETEC ENGINES or  LOOSE OR BROKEN ATTACHMENTS AND MISROUTED BATTERY CABLES COULD LEAD TO CABLE INSULATION DAMAGE",
    "Notification_Dates:for example like  OWNER NOTIFICATION BEGAN FEBRUARY 10, 2003.",
    "Corrective Action:for example like  DEALERS WILL INSPECT THE BATTERY CABLES FOR THE CONDITION OF THE CABLE INSULATION AND PROPER TIGHTENING OF THE TERMINAL ENDS or AS NECESSARY, CABLES WILL BE REROUTED, RETAINING CLIPS INSTALLED, AND DAMAGED BATTERY CABLES REPLACED.",
    "Contact_Information:for example like  1-866-436-7332. or SAFETY ADMINISTRATION'S AUTO SAFETY HOTLINE AT 1-888-DASH-2-DOT (1-888-327-4236).",
    "high risk: when really high risk causing heavy injury like brain damage/ribcage fracture/ legs broken etc ",
    "low risk:Low risk like small sprains/burns/bruises etc"
    "safety risk:airbag deployment failure, brake failure, acceleration issues, seatbelt malfunction, tire blowouts, electrical system failure, fuel system leaks, compromised structural integrity}"
]


TRIED TO IMPLEMENT CHAIN-OF THOUGHT

In [23]:
"""""
import typing
import torch
def get_fine_tune_prompt_json(rule_set: typing.List[str], input_str: str,label_str: str,tokenizer,) -> torch.Tensor:

  rule_str = "\n".join(rule_set)
  usr_msg1 ="You are a smart and intelligent Name Entity Recognition (NER) System. " \
           "The task involves using to extract and identify all instances of automotive domain entities " \
           "from a given dataset containing text related to the automotive domain. The identified entities should " \
           "be enclosed within JSON keys corresponding to their entity type in the output, which must have the same content as " \
           "the input. For instance, a car make should be represented as \"car_make\": \"<Make of a car>\"," \
           "while a car model should be represented as \"car_model\": \"<Model of a car>\". It is important " \
           "to ensure that all relevant automotive domain entities are identified, and false identifications are avoided." \
           f"""\n\nList Of Entities\n{rule_str}"""\
           "\n\n" \
           "Are the instructions clear to you?"

  asst_msg1 = "Yes, the instructions are clear. I will identify and associate each instance of the specified automotive domain entities "\
    "in the user utterance with their corresponding labels, enclosing them within JSON keys. For example, "\
    "\"car_make\": \"<Make of a car>\", \"car_model\": \"<Model of a car>\", etc. I will ensure that all relevant "\
    "automotive domain entities are detected accurately, without any false identifications."\

  usr_msg2 = "The engine cooling system may develop a leak due to a faulty gasket, resulting in coolant loss."\
     "This loss of coolant can lead to engine overheating, potentially causing severe damage to internal engine components."\


  asst_msg2 = """
     [{"Entity": "bottoming out the suspension", "Label": "Failure Issue"},
     {"Entity": "amplification of the stress", "Label": "Failure Issue"},
     {"Entity": "floor truss network", "Label": "Component"},
     {"Entity": "fracture of welds", "Label": "Failure Issue"},
     {"Entity": "chassis frame rail", "Label": "Component"},
     {"Entity": "floor truss network support system", "Label": "Component"},
     {"Entity": "damage to electrical wiring", "Label": "Failure Issue"},
     {"Entity": "fuel lines", "Label": "Component"},
     {"Entity": "fire", "Label": "Failure Issue"}]
        """



  usr_msg3 = "Give a brief explanation of why your answer is correct."

  asst_msg3 = """Sure, I have identified and enclosed within corresponding JSON keys,"""\
    """all instances of the specified automotive domain entities in the given text. For example,"""\
    """I identified the entity "chassis frame rail" as a component and enclosed it within the"""\
    """Entity" key, while assigning the corresponding "Label" key as "Component". The rest of"""\
    """the text was left unchanged as it did not contain any other identified automotive domain entities."""\
    """The output is a JSON object with the identified entities as values for the "Entity" key and their c"""\
    """corresponding labels as values for the "Label" key."""\

  usr_msg4 = "Alright! I am now going to give you another user utterance."\
    "Please detect automotive entities in it according to the previous instructions. Do"\
    "not include an explanation in your answer."\

  asst_msg4 = "Sure! Please give me the user utterance."



  messages = [
        {"role": "user", "content": usr_msg1},
        {"role": "assistant", "content": asst_msg1},
        {"role": "user", "content": usr_msg2},
        {"role": "assistant", "content": asst_msg2},
        {"role": "user", "content": usr_msg3},
        {"role": "assistant", "content": asst_msg3},
        {"role": "user", "content": usr_msg4},
        {"role": "assistant", "content": asst_msg4},
        {"role": "user", "content": input_str},
        {"role": "assistant", "content": label_str},
    ]

  encoded_input_ids = tokenizer.apply_chat_template(messages,tokenize=False)
  encoded_input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt")

  return encoded_input_ids

input_str="conditions can result in the bottoming out the suspension and amplification of the stress"\
    "placed on the floor truss network. the additional stress can result in the fracture of welds securing the"\
    "floor truss network system to the chassis frame rail and/or fracture of the floor truss network support"\
    "system. the possibility exists that there could be damage to electrical wiring and/or fuel lines which"\
    "could potentially lead to a fire."\

label_str="""
[ {"Entity": "bottoming out the suspension", "Label": "Failure Issue"},
{"Entity": "amplification of the stress", "Label": "Failure Issue"},
{"Entity": "floor truss network", "Label": "Component"},
{"Entity": "fracture of welds", "Label": "Failure Issue"},
{"Entity": "chassis frame rail", "Label": "Component"},
{"Entity": "floor truss network support system", "Label": "Component"},
{"Entity": "damage to electrical wiring", "Label": "Failure Issue"},
{"Entity": "fuel lines", "Label": "Component"},
{"Entity": "fire", "Label": "Failure Issue"} ]
"""
inputs=get_fine_tune_prompt_json(entlist,input_str , label_str , tokenizer)
inputs
outputs = model.generate(input_ids=inputs.to("cuda"), max_new_tokens=512)
print(tokenizer.batch_decode(outputs)[0])
"""""

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 51)

In [66]:
instr ="You are a smart and intelligent Name Entity Recognition (NER) System. " \
           "The task involves using to extract and identify all instances of automotive domain entities " \
           "from a given dataset containing text related to the automotive domain. The identified entities should " \
           "be enclosed within JSON keys corresponding to their entity type in the output, which must have the same content as " \
           """the input. For instance, a car  manufacturer should be represented as [{"Entity": "", "Label": "Failure Issue"}"""\
           "while a car model should be represented as \"car_model\": \"<Model of a car>\". It is important " \
           "to ensure that all relevant automotive domain entities are identified, and false identifications are avoided." \
           f"""\n\nHere are the List Of Entities\n{entlist}"""\
           "\n\n" \
           "I would want you to identify all instances of automotive domain entities in the below text from in the format like this "\
           """[{"Entity": "bottoming out the suspension", "Label": "Failure Issue"},
              {"Entity": "amplification of the stress", "Label": "Failure Issue"},
              {"Entity": "floor truss network", "Label": "Component"},"""\
           "with reference to the enitity list already given there should be No false idenitifications, dont give me any process just idenitfy the entities properly with no mistakes"\
           "With Each entity-label pair in a newline. Are the instructions clear to you?"\

text="If the key is not in the run position, the air bags may not deploy if the vehicle is involved in a crash,"\
     "increasing the risk of injury. Additionally, a key knocked out of the run position could cause loss of engine power,"\
     "power steering, and power braking, increasing the risk of a vehicle crash."\

prompt_template = lambda text: f'''[INST] {instr} \n{text} \n[/INST]'''
prompt = prompt_template(text)
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=400)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> [INST] You are a smart and intelligent Name Entity Recognition (NER) System. The task involves using to extract and identify all instances of automotive domain entities from a given dataset containing text related to the automotive domain. The identified entities should be enclosed within JSON keys corresponding to their entity type in the output, which must have the same content as the input. For instance, a car  manufacturer should be represented as [{"Entity": "", "Label": "Failure Issue"}while a car model should be represented as "car_model": "<Model of a car>". It is important to ensure that all relevant automotive domain entities are identified, and false identifications are avoided.

Here are the List Of Entities
['Manufacturer:like name of the car manufacturer', 'Model: for example like  a model like focus from manufacturer ford', 'Year: for example like made  in the  year 2000', 'Recall_Number:for example like  02V288000', 'Component: for example like ELECTRICAL SYSTEM or 

# **Hence required output from Few/Zero shot has been produced in the required json format**